# Lottery Project

Scraping Mega Millions website. Looking at the page's code using Chrome's Inspect tool, I find where relevant info is located in the page. I also note that there are a total of 94 pages to scrape. I begin with importing (potentially) necessary libraries and using a header that mimics my computer's.

In [1]:
from bs4 import BeautifulSoup
import requests
from requests import get
import urllib
import urllib.request

import pandas as pd
import numpy as np

In [2]:
h = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"
}

burl = "http://www.usamega.com/mega-millions-history.asp?p=1"
resp = get(burl,headers=h)
print(resp)

<Response [200]>


The response returning 200 means things are good to go. I begin by prototyping the scraper to pull the data I want. I test by scraping the first page.

In [3]:
#prototype
results = pd.DataFrame(columns=['d','s','m','multi','j'])
winnum = {}
req = urllib.request.Request(burl,headers=h)
with urllib.request.urlopen(req) as response:
    pg = response.read()
soup = BeautifulSoup(pg)

for row in soup('table')[4].findAll('tr'):
    tds=row('td')
    if tds[1].a is not None:
        winnum['d']=tds[1].a.string
        if tds[3].b is not None:
            nums = []
            numslist = tds[3].b.string.split(u"\u00B7")
            for num in numslist:
                nums.append(int(num))
            if tds[5].b is not None:
                winnum['multi']=tds[5].b.string
            if tds[7].a is not None:
                winnum['j']=tds[7].a.string
                
            winnum['s']=nums
            mnum=tds[3].strong.string #megaball
            winnum['m']=int(mnum)
            #print(winnum)
            results = results.append(winnum,ignore_index=True)
results

,d,s,m,multi,j
0,"Friday, October 16, 2020","[27, 32, 50, 52, 57]",12,5,$77 Million
1,"Tuesday, October 13, 2020","[11, 44, 45, 46, 70]",25,2,$69 Million
2,"Friday, October 9, 2020","[5, 11, 25, 27, 64]",13,2,$60 Million
3,"Tuesday, October 6, 2020","[15, 16, 18, 39, 59]",17,3,$50 Million
4,"Friday, October 2, 2020","[9, 38, 47, 49, 68]",25,2,$41 Million
5,"Tuesday, September 29, 2020","[14, 39, 43, 44, 67]",19,3,$32 Million
6,"Friday, September 25, 2020","[20, 36, 37, 48, 67]",16,2,$24 Million
7,"Tuesday, September 22, 2020","[36, 43, 44, 55, 68]",12,4,$22 Million
8,"Friday, September 18, 2020","[26, 29, 41, 52, 64]",11,3,$20 Million
9,"Tuesday, September 15, 2020","[25, 28, 38, 59, 62]",22,4,$119 Million


In [4]:
#scraping
results = pd.DataFrame(columns=['d','s','m','multi','j'])

i = 1
total = 94

while True:
    if i > total: break
    winnum = {}
    i = str(i)
    #soup = BeautifulSoup(urllib.urlopen("http://www.usamega.com/mega-millions-history.asp?p="+i).read())
    req = urllib.request.Request("http://www.usamega.com/mega-millions-history.asp?p="+i,headers=h)
    with urllib.request.urlopen(req) as response:
        pg = response.read()
    soup = BeautifulSoup(pg)
    
    
    for row in soup('table')[4].findAll('tr'):
        tds=row('td')
        if tds[1].a is not None:
            winnum['d']=tds[1].a.string
            if tds[3].b is not None:
                nums = []
                numslist = tds[3].b.string.split(u"\u00B7")
                for num in numslist:
                    nums.append(int(num))
                if tds[5].b is not None:
                    winnum['multi']=tds[5].b.string
                if tds[7].a is not None:
                    winnum['j']=tds[7].a.string
                
                winnum['s']=nums
                mnum=tds[3].strong.string #megaball
                winnum['m']=int(mnum)
                #print(winnum)
                results = results.append(winnum,ignore_index=True)
    i=int(i)
    i+=1
results

,d,s,m,multi,j
0,"Friday, October 16, 2020","[27, 32, 50, 52, 57]",12,5,$77 Million
1,"Tuesday, October 13, 2020","[11, 44, 45, 46, 70]",25,2,$69 Million
2,"Friday, October 9, 2020","[5, 11, 25, 27, 64]",13,2,$60 Million
3,"Tuesday, October 6, 2020","[15, 16, 18, 39, 59]",17,3,$50 Million
4,"Friday, October 2, 2020","[9, 38, 47, 49, 68]",25,2,$41 Million
...,...,...,...,...,...
2438,"Friday, October 4, 1996","[8, 16, 18, 36, 38]",1,None,$20 Million
2439,"Friday, September 27, 1996","[8, 25, 35, 37, 48]",8,None,$15 Million
2440,"Friday, September 20, 1996","[5, 24, 31, 34, 48]",6,None,$10 Million
2441,"Friday, September 13, 1996","[3, 4, 9, 30, 47]",1,None,$5 Million


In [5]:
#cleaning up dataframe
dates = results.d.str.split(",",n=1,expand=True)
results['DOW']=dates[0]
results['date']=dates[1]
results[['b1','b2','b3','b4','b5']] = pd.DataFrame(results.s.tolist())


In [6]:
results

,d,s,m,multi,j,DOW,date,b1,b2,b3,b4,b5
0,"Friday, October 16, 2020","[27, 32, 50, 52, 57]",12,5,$77 Million,Friday,"October 16, 2020",27,32,50,52,57
1,"Tuesday, October 13, 2020","[11, 44, 45, 46, 70]",25,2,$69 Million,Tuesday,"October 13, 2020",11,44,45,46,70
2,"Friday, October 9, 2020","[5, 11, 25, 27, 64]",13,2,$60 Million,Friday,"October 9, 2020",5,11,25,27,64
3,"Tuesday, October 6, 2020","[15, 16, 18, 39, 59]",17,3,$50 Million,Tuesday,"October 6, 2020",15,16,18,39,59
4,"Friday, October 2, 2020","[9, 38, 47, 49, 68]",25,2,$41 Million,Friday,"October 2, 2020",9,38,47,49,68
...,...,...,...,...,...,...,...,...,...,...,...,...
2438,"Friday, October 4, 1996","[8, 16, 18, 36, 38]",1,None,$20 Million,Friday,"October 4, 1996",8,16,18,36,38
2439,"Friday, September 27, 1996","[8, 25, 35, 37, 48]",8,None,$15 Million,Friday,"September 27, 1996",8,25,35,37,48
2440,"Friday, September 20, 1996","[5, 24, 31, 34, 48]",6,None,$10 Million,Friday,"September 20, 1996",5,24,31,34,48
2441,"Friday, September 13, 1996","[3, 4, 9, 30, 47]",1,None,$5 Million,Friday,"September 13, 1996",3,4,9,30,47


In [7]:
results.j

0       $77 Million
1       $69 Million
2       $60 Million
3       $50 Million
4       $41 Million
           ...     
2438    $20 Million
2439    $15 Million
2440    $10 Million
2441     $5 Million
2442     $5 Million
Name: j, Length: 2443, dtype: object

In [15]:
numbers = results.j.str.split(expand=True)
numbers

,0,1
0,$77,Million
1,$69,Million
2,$60,Million
3,$50,Million
4,$41,Million
...,...,...
2438,$20,Million
2439,$15,Million
2440,$10,Million
2441,$5,Million


Mega Millions number matrix was last changed on 10/31/2017. 